# Cleaning Data in SQL
Data from your queries may often contain missing values, duplicate rows, or data in the wrong format. It is crucial to be able to clean the data you retrieve through your queries. This template runs through the basics of handling missing data, duplicate rows, and string cleaning.

This template connects to a database containing different course datasets by default. However, you can use any integration you want, so long as you have access to a table with text data. You can learn more about integrations [here](https://workspace-docs.datacamp.com/integrations/what-is-an-integration).

💡&nbsp;&nbsp;_The data we will use is contained various schemas of the Course Databases database. To edit the queries, you must add the schema name when referencing tables in this database (e.g., `world.currencies`)._

## Missing values
Missing or null values are common in data and can often present a problem for future analyses. You can inspect the number of missing values in a column by filtering the data for null values and performing an aggregation. 

In the example below, we filter for records where the `unemployment_rate` is `NULL` and then `COUNT()` the number of rows.

In [1]:
SELECT COUNT(*) AS number_missing_unemployment_rates
FROM world.economies
WHERE unemployment_rate IS NULL

,number_missing_unemployment_rates
0,163


### Filling missing values
Once you have identified the missing values, you can handle them in several ways. 

Here, we use `COALESCE()` to replace NULL values with the average unemployment rate (which is accessed via a subquery). `COALESCE()` works by returning the first argument if it is not null. If it is null, it returns the second argument, and so forth. If the `unemployment_rate` column is `NULL`, it returns the second argument, which is the average unemployment we calculate with a subquery.

In [2]:
SELECT
	code,
    unemployment_rate,
	COALESCE(unemployment_rate, 
             (SELECT AVG(unemployment_rate) FROM world.economies)) AS filled_unemployment_rate
FROM world.economies

,code,unemployment_rate,filled_unemployment_rate
0,AFG,NaN,8.715097
1,AFG,NaN,8.715097
2,AGO,NaN,8.715097
3,AGO,NaN,8.715097
4,ALB,14.00,14.000000
...,...,...,...
375,ZAF,25.35,25.350000
376,ZMB,NaN,8.715097
377,ZMB,NaN,8.715097
378,ZWE,NaN,8.715097


## Duplicate rows
Another data issue you may come across is duplicate rows. To identify them, you can use `ROW_NUMBER()` to assign numbers to rows based on identical combinations. By choosing the `PARTITION` of the window function, you can specify over which columns you want to look for duplicates.

In the example below, we use `PARTITION BY` to assign row numbers based on the combination of country code and unemployment rate. As you can see from the query results, duplicate rows have a value of 2 or greater.

In [3]:
SELECT *
FROM (
    SELECT 
        code, 
        unemployment_rate,
        ROW_NUMBER() OVER(PARTITION BY code, unemployment_rate) AS row_number
    FROM world.economies
) AS sub

,code,unemployment_rate,row_number
0,AFG,NaN,1
1,AFG,NaN,2
2,AGO,NaN,1
3,AGO,NaN,2
4,ALB,14.00,1
...,...,...,...
375,ZAF,25.35,1
376,ZMB,NaN,1
377,ZMB,NaN,2
378,ZWE,NaN,1


### Discarding duplicate rows
Removing duplicate rows is just as simple as identifying them. To do so, you simply need to change your filter to select `row_number`s with a value of 1.

In [4]:
SELECT *
FROM (
    SELECT 
        code, 
        unemployment_rate,
        ROW_NUMBER() OVER(PARTITION BY code, unemployment_rate) AS row_number
    FROM world.economies
) AS sub
WHERE row_number = 1

,code,unemployment_rate,row_number
0,AFG,NaN,1
1,AGO,NaN,1
2,ALB,14.000,1
3,ALB,17.100,1
4,ARE,NaN,1
...,...,...,...
293,YEM,NaN,1
294,ZAF,24.875,1
295,ZAF,25.350,1
296,ZMB,NaN,1


## Invalid data
If you are aware of invalid data, you can find (and remove) it using pattern-matching.

In the example below, we search for rows where the `indep_year` contains a negative value. To do so, we convert the column to text using `::TEXT`, and then use `LIKE` and our pattern. The pattern we use searches for a minus sign (`-`), followed by any other characters (using the wildcard `%`).

In [5]:
SELECT indep_year
FROM world.countries
WHERE indep_year::TEXT LIKE '-%'

,indep_year
0,-1000
1,-660
2,-1523


You can also use pattern matching to find rows with similar variants. In the example below, we use a pattern to identify all rows with `Monarchy` in the `gov_form` column.

Our pattern searches for any row with "Monarchy". We use the `%` wildcard characters to allow for words/whitespace on either side of the word we are searching for.

In [6]:
SELECT DISTINCT name, gov_form
FROM world.countries
WHERE gov_form LIKE '%Monarchy%'

,name,gov_form
0,Sweden,Constitutional Monarchy
1,Tonga,Monarchy
2,Liechtenstein,Constitutional Monarchy
3,Jordan,Constitutional Monarchy
4,Solomon Islands,Constitutional Monarchy
5,Malaysia,"Constitutional Monarchy, Federation"
6,Saint Kitts and Nevis,Constitutional Monarchy
7,Norway,Constitutional Monarchy
8,Saint Vincent and the Grenadines,Constitutional Monarchy
9,Australia,"Constitutional Monarchy, Federation"


### Fixing invalid data
There are a variety of ways you can fix invalid data. One way is to use a `CASE` statement to recategorize the data. In the example below, we convert all `gov_form` rows that contain "Monarchy" to "Monarchy". The remaining entries are left as they are.

In [7]:
SELECT DISTINCT 
	name, 
    gov_form,
    CASE WHEN gov_form LIKE '%Monarchy%' THEN 'Monarchy' 
    ELSE gov_form END AS fixed_gov_form
FROM world.countries
WHERE gov_form LIKE '%Monarchy%'

,name,gov_form,fixed_gov_form
0,Saudi Arabia,Monarchy,Monarchy
1,United Kingdom,Constitutional Monarchy,Monarchy
2,Denmark,Constitutional Monarchy,Monarchy
3,Qatar,Monarchy,Monarchy
4,Brunei,Monarchy (Sultanate),Monarchy
5,Sweden,Constitutional Monarchy,Monarchy
6,Jamaica,Constitutional Monarchy,Monarchy
7,Antigua and Barbuda,Constitutional Monarchy,Monarchy
8,Samoa,Parlementary Monarchy,Monarchy
9,Australia,"Constitutional Monarchy, Federation",Monarchy


## Data types
You can query the `columns` table of the `information_schema` database to learn more about the data types of the table you will be working with.

In the query below, we retrieve each column and the data type for the `rental` table in the `dvdrentals` schema.

In [8]:
SELECT 
	column_name,
    data_type
FROM information_schema.columns
WHERE table_name = 'rental'

,column_name,data_type
0,rental_id,smallint
1,rental_date,timestamp without time zone
2,inventory_id,integer
3,customer_id,smallint
4,return_date,timestamp without time zone
5,staff_id,smallint
6,last_update,timestamp without time zone


### Converting data types
Sometimes, a column may not be the correct data type. You can use `CAST()` or `::` to convert a column to a specified type.

In the example below, we convert two strings and two integers to different data types. The latter two columns produce identical results. The column `integer_to_text` converts the integer 16 to text using `CAST()`. The column `integer_to_text_with_operator` does the same with the cast operator `::`.

In [9]:
SELECT
	CAST('42' AS INTEGER) AS string_to_integer,
    CAST('2022-06-01' AS DATE) AS string_to_date,
    CAST(16 AS TEXT) AS integer_to_text,
    16::TEXT AS integer_to_text_with_operator

,string_to_integer,string_to_date,integer_to_text,integer_to_text_with_operator
0,42,2022-06-01 00:00:00+00:00,16,16


### Converting date formats
Sometimes you may want to convert a date into a new format. You can use `TO_CHAR()` to convert a given date to a provided format.

In the example below, we use the short name of the month and the last two digits of the year to convert the precise rental date to a month_year column.

_Note: In Workspace, SQL queries are converted to pandas DataFrames. As a result, some formatting strings may result in Python automatically interpreting the result as a datetime and converting the date back to the original format._

In [10]:
SELECT 
	rental_id, 
    rental_date, 
    TO_CHAR(rental_date, 'Mon-YY') AS month_year
FROM dvdrentals.rental
LIMIT 5

,rental_id,rental_date,month_year
0,1,2005-05-24 22:53:30+00:00,May-05
1,2,2005-05-24 22:54:33+00:00,May-05
2,3,2005-05-24 23:03:39+00:00,May-05
3,4,2005-05-24 23:04:41+00:00,May-05
4,5,2005-05-24 23:05:21+00:00,May-05


## Next steps
Want to learn more techniques for cleaning data in PostgreSQL? Check out [Cleaning Data in PostgreSQL Databases](https://app.datacamp.com/learn/courses/cleaning-data-in-postgresql-databases)!

Alternatively, if you are interested in applying these skills to other SQL databases, check out our [sample integrations](https://app.datacamp.com/workspace/datasets?selectedLabels=%5B%22sql%22%5D) here!